# Collaboration and Competition

---

Congratulations for completing the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control agents in a more challenging environment, where the goal is to train a team of agents to play soccer.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Soccer.app"`
- **Windows** (x86): `"path/to/Soccer_Windows_x86/Soccer.exe"`
- **Windows** (x86_64): `"path/to/Soccer_Windows_x86_64/Soccer.exe"`
- **Linux** (x86): `"path/to/Soccer_Linux/Soccer.x86"`
- **Linux** (x86_64): `"path/to/Soccer_Linux/Soccer.x86_64"`
- **Linux** (x86, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86"`
- **Linux** (x86_64, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86_64"`

For instance, if you are using a Mac, then you downloaded `Soccer.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Soccer.app")
```

In [3]:
# env = UnityEnvironment(file_name="Soccer.app")
env = UnityEnvironment(file_name='/usr/src/app/Soccer_Linux_NoVis/Soccer.x86_64', no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 2
        Number of External Brains : 2
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: GoalieBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Unity brain name: StrikerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 6
        Vector Action descriptions: , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we obtain separate brains for the striker and goalie agents.

In [7]:
# print the brain names
print(env.brain_names)

# set the goalie brain
g_brain_name = env.brain_names[0]
g_brain = env.brains[g_brain_name]

# set the striker brain
s_brain_name = env.brain_names[1]
s_brain = env.brains[s_brain_name]

['GoalieBrain', 'StrikerBrain']


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)

# number of agents 
num_g_agents = len(env_info[g_brain_name].agents)
print('Number of goalie agents:', num_g_agents)
num_s_agents = len(env_info[s_brain_name].agents)
print('Number of striker agents:', num_s_agents)

# number of actions
g_action_size = g_brain.vector_action_space_size
print('Number of goalie actions:', g_action_size)
s_action_size = s_brain.vector_action_space_size
print('Number of striker actions:', s_action_size)

# examine the state space 
g_states = env_info[g_brain_name].vector_observations
g_state_size = g_states.shape[1]
print('There are {} goalie agents. Each receives a state with length: {}'.format(g_states.shape[0], g_state_size))
s_states = env_info[s_brain_name].vector_observations
s_state_size = s_states.shape[1]
print('There are {} striker agents. Each receives a state with length: {}'.format(s_states.shape[0], s_state_size))

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(2):                                         # play game for 2 episodes
    env_info = env.reset(train_mode=False)                 # reset the environment    
    g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
    s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
    g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
    s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)
    while True:
        # select actions and send to environment
        g_actions = np.random.randint(g_action_size, size=num_g_agents)
        s_actions = np.random.randint(s_action_size, size=num_s_agents)
        actions = dict(zip([g_brain_name, s_brain_name], 
                           [g_actions, s_actions]))
        env_info = env.step(actions)                       
        
        # get next states
        g_next_states = env_info[g_brain_name].vector_observations         
        s_next_states = env_info[s_brain_name].vector_observations
        
        # get reward and update scores
        g_rewards = env_info[g_brain_name].rewards  
        s_rewards = env_info[s_brain_name].rewards
        g_scores += g_rewards
        s_scores += s_rewards
        
        # check if episode finished
        done = np.any(env_info[g_brain_name].local_done)  
        
        # roll over states to next time step
        g_states = g_next_states
        s_states = s_next_states
        
        # exit loop if episode finished
        if done:                                           
            break
    print('Scores from episode {}: {} (goalies), {} (strikers)'.format(i+1, g_scores, s_scores))

When finished, you can close the environment.

In [ ]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import numpy as np
import random
import datetime
import copy
from collections import namedtuple, deque

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ",device)

In [ ]:
class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=1024, fc2_units=256): 
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
            fc3_units (int): Number of nodes in third hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        return self.fc3(x)

In [1]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, config, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            config (dict): constants for the agent
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.config = config
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.config['LR'])

        # Replay memory
        self.memory = ReplayBuffer(action_size, self.config['BUFFER_SIZE'], self.config['BATCH_SIZE'], seed)

        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done, step):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # If enough samples are available in memory, get random subset and learn
        if len(self.memory) > self.config['BATCH_SIZE']:
            experiences = self.memory.sample()
            self.learn(experiences, self.config['GAMMA'])

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()

        with torch.no_grad():
            action_values = self.qnetwork_local(state)

        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, self.config['TAU'])                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
    

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""

        return len(self.memory)


In [2]:
class MADDPG:
    def __init__(self, agent_config, model_config, random_seed):
        super(MADDPG, self).__init__()
        
        self.n_g_agents = agent_config["n_g_agents"]
        self.n_s_agents = agent_config["n_s_agents"]
        self.g_action_size = agent_config["g_action_size"]
        self.s_action_size = agent_config["s_action_size"]
        self.g_state_size = agent_config["g_state_size"]
        self.s_state_size = agent_config["s_state_size"]

        self.g_agents = [Agent(model_config, self.g_state_size, self.g_action_size, random_seed) 
                            for _ in range(self.n_g_agents)]
        self.s_agents = [Agent(model_config, self.s_state_size, self.s_action_size, random_seed) 
                            for _ in range(self.n_s_agents)]
                
    def reset(self):
        [agent.reset() for agent in self.g_agents]
        [agent.reset() for agent in self.s_agents]
        
    def g_act(self, states, noise=True):
        """get actions from all agents in the MADDPG object"""
        return [agent.act(state) for agent, state in zip(self.g_agents, states)]

    def s_act(self, states, noise=True):
        """get actions from all agents in the MADDPG object"""
        return [agent.act(state) for agent, state in zip(self.s_agents, states)]
    
    def g_step(self, state, action, reward, next_state, done, step):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        for idx, agent in enumerate(self.g_agents):
            agent.step(state[idx], action[idx], reward[idx], next_state[idx], int(done), step)

    def s_step(self, state, action, reward, next_state, done, step):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        for idx, agent in enumerate(self.s_agents):
            agent.step(state[idx], action[idx], reward[idx], next_state[idx], int(done), step)
  
    def g_save_agents(self):
        for n, agent in enumerate(self.g_agents):
            torch.save(agent.actor_local.state_dict(), 'models/goalie_actor_local_{}.pth'.format(n))
            torch.save(agent.critic_local.state_dict(), 'models/goalie_critic_local_{}.pth'.format(n))
            torch.save(agent.actor_target.state_dict(), 'models/goalie_actor_target_{}.pth'.format(n))
            torch.save(agent.critic_target.state_dict(), 'models/goalie_critic_target_{}.pth'.format(n))
            
    def s_save_agents(self):
        for n, agent in enumerate(self.s_agents):
            torch.save(agent.actor_local.state_dict(), 'models/goalie_actor_local_{}.pth'.format(n))
            torch.save(agent.critic_local.state_dict(), 'models/goalie_critic_local_{}.pth'.format(n))
            torch.save(agent.actor_target.state_dict(), 'models/goalie_actor_target_{}.pth'.format(n))
            torch.save(agent.critic_target.state_dict(), 'models/goalie_critic_target_{}.pth'.format(n))

In [3]:
agent_config = {"n_g_agents": num_g_agents,
                "n_s_agents": num_s_agents,
                "g_action_size": g_action_size,
                "s_action_size": s_action_size,
                "g_state_size": g_state_size,
                "s_state_size": s_state_size}

model_config = {
'BUFFER_SIZE': int(1e5),  # replay buffer size
'BATCH_SIZE': 64,         # minibatch size
'GAMMA': 0.99,            # discount factor
'TAU': 1e-3,              # for soft update of target parameters
'LR': 5e-4,               # learning rate 
'UPDATE_EVERY': 4,        # how often to update the network
'eps_start': 1.0, 
'eps_end': 0.01, 
'eps_decay': 0.995
}

NameError: name 'num_g_agents' is not defined

In [ ]:
m_agent = MADDPG(agent_config=agent_config, model_config=model_config, random_seed=2)

In [ ]:
def mddpg(n_episodes=5000, time_window=100):

    scores_deque_one = deque(maxlen=100)
    scores_deque_two = deque(maxlen=100)
    
    global_scores_one = []
    global_scores_two = []

    start_time = datetime.datetime.now().replace(microsecond=0)
    
    for i_episode in range(1, n_episodes+1):

        step = 0

        env_info = env.reset(train_mode=True)      # reset the environment    

        g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
        s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
        g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
        s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)
                
        while True:

            g_actions = m_agent.g_act(g_states)
            s_actions = m_agent.s_act(s_states)

            actions = dict(zip([g_brain_name, s_brain_name], 
                               [g_actions, s_actions]))
        
            env_info = env.step(actions)           # send all actions to tne environment

            # get next states
            g_next_states = env_info[g_brain_name].vector_observations         
            s_next_states = env_info[s_brain_name].vector_observations

            g_rewards = env_info[g_brain_name].rewards  
            s_rewards = env_info[s_brain_name].rewards
            
            # see if episode finished
            g_dones = np.any(env_info[g_brain_name].local_done)   
            s_dones = np.any(env_info[s_brain_name].local_done)        

            m_agent.g_step(g_states, g_actions, g_rewards, g_next_states, g_dones, step)
            m_agent.s_step(s_states, s_actions, s_rewards, s_next_states, s_dones, step)

            step += 1

            # roll over states to next time step
            g_states = g_next_states
            s_states = s_next_states

            g_scores += g_rewards
            s_scores += s_rewards

            if np.any([g_dones, s_dones]):                  # exit loop if episode finished
                break

        mean_score_one = np.around(np.mean([g_scores[0], s_scores[0]]), decimals=3)
        mean_score_two = np.around(np.mean([g_scores[1], s_scores[1]]), decimals=3)

        global_scores_one.append(mean_score_one)
        global_scores_two.append(mean_score_two)

        scores_deque_one.append(mean_score_one)
        scores_deque_two.append(mean_score_two)
        
        g_scores = np.around(g_scores, decimals=3)
        s_scores = np.around(s_scores, decimals=3)

        average_score_one = np.around(np.mean(scores_deque_one), decimals=3)
        average_score_two = np.around(np.mean(scores_deque_two), decimals=3)

        print('\rCurrent Episode: {}, Average One: {:.2f}, Average Two: {:.2f}, Goalies Score: {}, Striker Score: {}'\
              .format(i_episode, mean_score_one, mean_score_two, g_scores, s_scores), end="")

            
        if i_episode % time_window == 0 or (len(scores_deque_one) == 100 and \
                                            (np.max(average_score_one, average_score_two) >= 10.0)):  

            current_time = datetime.datetime.now().replace(microsecond=0)
            time_difference = current_time - start_time
            print('\rAverage Episode {}\t Average One {:.2f}, \t Average Two {:.2f}\tTime: {}'\
                  .format(i_episode, average_score_one, average_score_two, time_difference)) 

        if len(scores_deque_one) == 100 and (np.max(average_score_one, average_score_two) >= 10.0):  
            print('Environment solved!')
            m_agent.s_save_agents()
            m_agent.g_save_agents()

    return global_scores_one, global_scores_two, m_agent

In [ ]:
scores_one, scores_two, m_agent = mddpg()

In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores_one)), scores_one)
plt.axhline(y=0.5, linewidth=2, color = 'r')
plt.xlabel('Episode number')
plt.ylabel('Score')
plt.show()

In [ ]:
# eval_m_agent = MADDPG(state_size=state_size, action_size=action_size, num_agents=num_agents, agent_config=agent_config, random_seed=1111)

# for n, agent in enumerate(eval_m_agent.ddpg_agents):
    
#     agent.actor_local.load_state_dict(torch.load('models/model_actor_local_{}.pth'.format(n), map_location=device.type), strict=False)
#     agent.critic_local.load_state_dict(torch.load('models/model_critic_local_{}.pth'.format(n), map_location=device.type), strict=False)
#     agent.actor_target.load_state_dict(torch.load('models/model_actor_target_{}.pth'.format(n), map_location=device.type), strict=False)
#     agent.critic_target.load_state_dict(torch.load('models/model_critic_target_{}.pth'.format(n), map_location=device.type), strict=False)

In [ ]:
# env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
# states = env_info.vector_observations
# eval_scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# score_average = 0# get the current state (for each agent)

# while True:
#     actions = eval_m_agent.act(states, noise=False)
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished

#     states = next_states                               # roll over states to next time step
#     eval_scores += rewards                                  # update the score (for each agent)    
    
#     if np.any(dones):                                  # exit loop if episode finished
#         break